In [ ]:
#| default_exp core

# Experimental Design

> Design experiments in Python.

In [ ]:
#| hide
from nbdev.showdoc import *

k = #factors

L = #levels

n = #replications

N = #runs = k * L * n

In [ ]:
#| export
import numpy as np
import pandas as pd

from fastcore.basics import patch

In [ ]:
#| export
class ExperimentalDesign():
    """Class to hold and manage the Experimental Design matrix"""
    def __init__(
        self,
        design: pd.DataFrame, # base design matrix
        factors: list, # list of experimental factors
        center_vals: list, # list of center values for each factors
        delta_vals: list # list of delta values for each factor
    ):
        self.design: pd.DataFrame = design
        self.factors: list = factors
        self.model_matrix: np.ndarray = design.query("rep==0")[factors].values
        self.experimental_matrix = None
        
        if center_vals and delta_vals:
            self.centers = dict(zip(factors, center_vals))
            self.deltas = dict(zip(factors, delta_vals))
            self.experimental_matrix = self.create_experimental_matrix()
    
    def __repr__(self):
        if self.experimental_matrix is not None:
            return f"{self.experimental_matrix}"
        else:
            return f"{self.design}"
    
    def __str__(self):
        return self.__repr__()
    
    def create_experimental_matrix(self) -> pd.DataFrame:
        """Applies level values to the model matrix"""
        experiment_design = self.design.copy()
        for factor in self.factors:
            experiment_design = experiment_design.assign(
                **{factor: self.design[factor] * self.deltas[factor] + self.centers[factor]}
            )
        return experiment_design

In [ ]:
show_doc(ExperimentalDesign.create_experimental_matrix)

---

#### ExperimentalDesign.create_experimental_matrix

>      ExperimentalDesign.create_experimental_matrix ()

Applies level values to the model matrix

In [ ]:
factors = ["a", "b"]
design = pd.DataFrame({
    "a": [-1, 1],
    "b": [1, -1],
    "rep": [0, 0]
})
ed = ExperimentalDesign(design, factors, center_vals=[0.5, 2], delta_vals=[0.1, 1]); ed

     a  b  rep
0  0.4  3    0
1  0.6  1    0

In [ ]:
#| export

@patch
def calc_orthogonality(self: ExperimentalDesign) -> int:
    """Calculate the orthogonality of the model matrix"""
    return self.model_matrix.sum(axis=1).sum()

In [ ]:
ed.calc_orthogonality()

0

In [ ]:
#| export

@patch
def is_orthogonal(self: ExperimentalDesign) -> bool:
    """Checks orthogonality of the model matrix"""
    return self.calc_orthogonality() == 0
    

In [ ]:
ed.is_orthogonal()

True

In [ ]:
#| hide
from nbdev import nbdev_export; nbdev_export()